In [1]:
!pip install pyspark==3.2.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 794.6 kB/s eta 0:00:00m eta 0:00:010:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 KB 1.1 MB/s eta 0:00:000:00:01
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805910 sha256=700cf3597aece4d4d33985cff86ea722cb4828abc92d6a43191eda2f3b037411
  Stored in directory: /home/majid/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark


In [5]:
!pip install  findspark

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from pyspark import SparkContext
sc = SparkContext(master = 'local' )


In [6]:
import findspark
findspark.init()

In [3]:
# set java home path if not set
import os
os.environ['JAVA_HOME'] = 'xxxx'

In [9]:
# inspect SparkContext
print("version",sc.version)   #Retrieve SparkContext version
print("pythonVer",sc.pythonVer) #Retrieve Python version
print("master",sc.master)  #Master URL to connect to
print('spark home',str(sc.sparkHome))  #Path where Spark is installed on worker nodes
print("user",str(sc.sparkUser())) #Retrieve name of the Spark User running SparkContext
print("appname",sc.appName ) #Return application name
print("appid",sc.applicationId)  #Retrieve application ID
print("level of parallelism",sc.defaultParallelism) #Return default level of parallelism
print("defaultMinPartitions",sc.defaultMinPartitions) #Default minimum number of partitions forRDDs

version 3.2.0
pythonVer 3.10
master local
spark home None
user majid
appname pyspark-shell
appid local-1711400423228
level of parallelism 1
defaultMinPartitions 1


In [10]:
sc.stop()

In [11]:
# Configuration
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
 .setMaster("local")
 .setAppName("appname" )
 .set("spark.executor.memory" , "1g" ))

sc = SparkContext(conf = conf)

In [12]:
# Parallelized Collections
# rdd is a Resilient Distributed Dataset
rdd = sc.parallelize([('a',7),('a',2),('b',2)])
rdd2 = sc.parallelize([('a',2),('d',1),('b',1)])
rdd3 = sc.parallelize(range(100))
rdd4 = sc.parallelize([("a" ,["x" ,"y" ,"z" ]),("b" ,["p" ,"r" ])])

In [13]:
print("List the number of partitions",rdd.getNumPartitions())
print("Count RDD instances",str(rdd.count()))
print("Count RDD instances by key",str(rdd.countByKey()))

List the number of partitions 1


Count RDD instances 3
Count RDD instances by key defaultdict(<class 'int'>, {'a': 2, 'b': 1})


In [36]:
print("Count RDD instances by value",str(rdd.countByValue()))
print("Return (key,value) pairs as a dictionary",str(rdd.collectAsMap()))
print("Sum of RDD elements",str(rdd3.sum()))
print("Check whether RDD is empty",str(sc.parallelize([]).isEmpty()))

Count RDD instances by value defaultdict(<class 'int'>, {('a', 7): 1, ('a', 2): 1, ('b', 2): 1})
Return (key,value) pairs as a dictionary {'a': 2, 'b': 2}
Sum of RDD elements 4950
Check whether RDD is empty True


In [16]:
# summary
print('max',rdd3.max()) #Maximum value of RDD elements 99
print('min',rdd3.min()) #Minimum value of RDD elements0
print('mean',rdd3.mean()) #Mean value of RDD elements 49.5
print('stdev',rdd3.stdev()) #Standard deviation of RDD elements 28.866070047722118
print('variance',rdd3.variance()) #Compute variance of RDD elements 833.25
print('histogram',rdd3.histogram(3)) #Compute histogram by bins([0,33,66,99],[33,33,34])
print('stats',rdd3.stats()) #Summary statistics (count, mean, stdev, max & min)


max 99
min 0
mean 49.5
stdev 28.86607004772212
variance 833.25
histogram ([0, 33, 66, 99], [33, 33, 34])
stats (count: 100, mean: 49.5, stdev: 28.86607004772212, max: 99, min: 0)


In [18]:
# flat map is used to flatten the result
# map is used to apply a function to each RDD element
# collect is used to collect the result


#Apply a function to each RDD element and flatten the result>>>
rdd5 = rdd.flatMap(lambda x: x+(x[1],x[0]))
rdd5.collect()
#['a',7 , 7 ,  'a' , 'a' , 2,  2,  'a', 'b', 2 , 2, 'b']

#Apply a flatMap function to each (key,value) pair of rdd4 without changing the keys
rdd4.flatMapValues(lambda x: x).collect()
#[('a', 'x'), ('a', 'y'), ('a', 'z'),('b', 'p'),('b', 'r')]

[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

In [19]:
#Apply a function to each RFD element
rdd.map(lambda x: x +(x[1],x[0])).collect()

[('a', 7, 7, 'a'), ('a', 2, 2, 'a'), ('b', 2, 2, 'b')]

In [20]:
rdd5 = rdd.flatMap(lambda x: x +(x[1],x[0]))
rdd5.collect()

['a', 7, 7, 'a', 'a', 2, 2, 'a', 'b', 2, 2, 'b']

In [21]:
rdd4.flatMapValues(lambda x: x).collect()

[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

In [22]:
# Getting
print("Return a list with all RDD elements",rdd.collect())
print("Return the first element of the RDD",rdd.first())
print("Return the first n elements of the RDD",rdd.take(2))
print("Return the top n elements of the RDD",rdd.top(2))


Return a list with all RDD elements [('a', 7), ('a', 2), ('b', 2)]
Return the first element of the RDD ('a', 7)
Return the first n elements of the RDD [('a', 7), ('a', 2)]
Return the top n elements of the RDD [('b', 2), ('a', 7)]


In [23]:
# sampling
rdd3.sample(False, 0.15, 81).collect() 
#Return sampled subset of rdd3     [3,4,27,31,40,41,42,43,60,76,79,80,86,97]

[3, 4, 27, 28, 35, 41, 43, 49, 53, 58, 85, 93]

In [24]:
# Filtering
rdd.filter(lambda x: "a" in x).collect() #Filter the RDD[('a',7),('a',2)]
rdd5.distinct().collect() #Return distinct RDD values['a' ,2, 'b',7]
rdd.keys().collect() #Return (key,value) RDD's keys['a',  'a',  'b']

['a', 'a', 'b']

In [25]:
def g (x): print(x)

rdd.foreach(g) #Apply a function to all RDD elements('a', 7)('b', 2)('a', 2)

('a', 7)
('a', 2)
('b', 2)


In [27]:
# Reducing
rdd.reduceByKey(lambda x,y : x+y).collect()
 #Merge the rdd values for each key[('a',9),('b',2)]

rdd.reduce(lambda a, b: a+ b) #Merge the rdd values('a', 7, 'a' , 2 , 'b' , 2)

('a', 7, 'a', 2, 'b', 2)

In [30]:
# Grouping by
rdd3.groupBy(lambda x: x % 2) \
.mapValues(list).collect() #Return RDD of grouped values 



[(0,
  [0,
   2,
   4,
   6,
   8,
   10,
   12,
   14,
   16,
   18,
   20,
   22,
   24,
   26,
   28,
   30,
   32,
   34,
   36,
   38,
   40,
   42,
   44,
   46,
   48,
   50,
   52,
   54,
   56,
   58,
   60,
   62,
   64,
   66,
   68,
   70,
   72,
   74,
   76,
   78,
   80,
   82,
   84,
   86,
   88,
   90,
   92,
   94,
   96,
   98]),
 (1,
  [1,
   3,
   5,
   7,
   9,
   11,
   13,
   15,
   17,
   19,
   21,
   23,
   25,
   27,
   29,
   31,
   33,
   35,
   37,
   39,
   41,
   43,
   45,
   47,
   49,
   51,
   53,
   55,
   57,
   59,
   61,
   63,
   65,
   67,
   69,
   71,
   73,
   75,
   77,
   79,
   81,
   83,
   85,
   87,
   89,
   91,
   93,
   95,
   97,
   99])]

In [31]:
rdd.groupByKey() \
.mapValues(list).collect()        
 # [('a',[7,2]),('b',[2])]

[('a', [7, 2]), ('b', [2])]

In [32]:
# Aggregating
seqOp = (lambda x,y: (x[0]+y,x[1]+1))
combOp = (lambda x,y:(x[0]+y[0],x[1]+y[1]))
#Aggregate RDD elements of each partition and then the results
rdd3.aggregate((0,0),seqOp,combOp) #(4950,100)
#Aggregate values of each RDD key>>> 
rdd.aggregateByKey((0,0),seqOp,combOp).collect()      #[('a',(9,2)), ('b',(2,1))]
#Aggregate the elements of each partition, and then the results>>> rdd3.fold(0,add)     4950#Merge the values for each key>>> rdd.foldByKey(0, add).collect()[('a' ,9), ('b' ,2)]#Create tuples of RDD elements by applying a function>>> rdd3.keyBy(lambda x: x+x).collect()

[('a', (9, 2)), ('b', (2, 1))]

In [33]:
print('rdd',rdd.collect())
print('rdd2',rdd2.collect())
print('substract rdd from rdd2',rdd.subtract(rdd2).collect())
rdd.subtract(rdd2).collect()
#Return each rdd value not contained in rdd2[('b' ,2), ('a' ,7)]
#Return each (key,value) pair of rdd2 with no matching key in rdd
print('subtractByKey',rdd2.subtractByKey(rdd).collect())
print('cartesian',rdd.cartesian(rdd2).collect()) #Return the Cartesian product of rdd and rdd2

rdd [('a', 7), ('a', 2), ('b', 2)]
rdd2 [('a', 2), ('d', 1), ('b', 1)]
substract rdd from rdd2 [('a', 7), ('b', 2)]
subtractByKey [('d', 1)]
cartesian [(('a', 7), ('a', 2)), (('a', 7), ('d', 1)), (('a', 7), ('b', 1)), (('a', 2), ('a', 2)), (('a', 2), ('d', 1)), (('a', 2), ('b', 1)), (('b', 2), ('a', 2)), (('b', 2), ('d', 1)), (('b', 2), ('b', 1))]


In [34]:
print('sort by value', rdd2.sortBy(lambda x: x[1]).collect())
 #Sort RDD by given function[('d',1),('b',1),('a',2)]
print('sort by value', rdd2.sortByKey().collect() ) #Sort (key, value) ROD by key[('a' ,2), ('b' ,1), ('d' ,1)]

sort by value [('d', 1), ('b', 1), ('a', 2)]
sort by value [('a', 2), ('b', 1), ('d', 1)]


In [35]:
rdd.repartition(4) #New RDD with 4 partitions
rdd.coalesce(1) #Decrease the number of partitions in the RDD to 1

CoalescedRDD[104] at coalesce at NativeMethodAccessorImpl.java:0

In [68]:
#rdd.saveAsTextFile("rdd.txt") 
# rdd.saveAsHadoopFile("hdfs:// namenodehost/parent/child",
# 'org.apache.hadoop.mapred.TextOutputFormat')

In [37]:
sc.stop()